In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import copy
import scipy.stats as sps
from sympy import *
import skgof
from matplotlib import cm
plt.style.use('ggplot')
%matplotlib inline

def scan_from_csv(filename,header=None):
    if not header:
        return pd.read_csv(filename,na_values='None')
    else:
        return pd.read_csv(filename,na_values='None',header=-1)

def write_answer_to_file(answer,file):
    with open(file, 'w') as answer_file:  
        answer_file.write(answer)

In [2]:
dataF = scan_from_csv('slump_test.data.txt')
dataF.head()

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82


In [3]:
data = np.array(dataF[['Water', 'SP', 'Fine Aggr.', 'SLUMP(cm)', 'FLOW(cm)', 'Compressive Strength (28-day)(Mpa)']])

In [4]:
data.shape

(103, 6)

Идея критерия: 

По обучающей выборке оценим $\mu, \Sigma$.

Теперь проверим тестовую выборку на соответствие гауссовскому распределению $\mathcal{N} (\mu, \Sigma)$.

Для этого сгенерируем $k$ векторов $v$ размерности распределения(как либо случайно). Далее, если $x \sim \mathcal{N} (\mu, \Sigma)$, то

$(v,x) \sim  \mathcal{N}(v\mu, v \Sigma v^T)$ - то есть одномерное нормальное с известными параметрами или их оценками. 

Для проверки принадлежности к этому распределению просто воспользуемся критерием Колмогорова-Смирнова.

Сгенерировав много $k$ получим много гипотез, проверим сложную гипотезу, что исходное распределение многомерное нормальное.

Для проверки сложной гипотезы будем использовать нисходящий метод Холма.

Рассмотрим проверку гипотезы на уровне $\alpha = 0.05$ 

In [5]:
from statsmodels.sandbox.stats.multicomp import multipletests  # Holm method
from skgof import ks_test

In [164]:
T = 100  # check T simple hypotheses
r = int(len(data)*0.6)
np.random.seed(42)
inds = np.arange(len(data))
np.random.shuffle(inds)
Train = np.array([data[i] for i in  inds[:r]])
Test = np.array([data[i] for i in inds[r:]])
K = sps.norm.rvs(size=(T,len(data[0])))  # random vectors for projections

In [165]:
t_mean = np.matrix(np.mean(Train,axis=0))
t_cov = np.matrix(np.cov(Train.T))

In [166]:
def test_proj(X,v):
    X = np.matrix(X)
    v = np.matrix(v)
    X_proj = np.array(v*X.T)[0]
    loc = np.array(t_mean*v.T)[0]
    scale = (np.array(v*t_cov*v.T)[0])**0.5
    return ks_test(X_proj, sps.norm(loc=loc,scale=scale))[1]

In [167]:
run_criteries = [
    (lambda X,v: test_proj(X,v)),
]

In [168]:
alphas = [0.05]
for a in alphas:
    p_values = []
    for t in range(T):
        for ex in run_criteries:
            p_values.append(ex(Test,K[t]))
    reject = multipletests(pvals=p_values, method='holm',alpha=a)[0]
    print('for alpha = ' + str(a) + ' : ' + str(np.sum(reject)) + ' simple hypotheses are rejected:')
    print('p_values: ' + str(p_values))
    print('reject_bits: ' + str(reject))
    print()    

for alpha = 0.05 : 0 simple hypotheses are rejected:
p_values: [0.835330268499336, 0.52115529210960221, 0.70017413231490955, 0.61790677126870142, 0.67146268117530905, 0.81547002134232949, 0.50289622547967805, 0.52352640019297569, 0.95885604478368902, 0.84423903516784082, 0.45508573388554652, 0.35682518352425852, 0.54738326167308327, 0.61561197963846892, 0.22833237741626511, 0.56434188460155044, 0.47372140690755937, 0.3057640934084197, 0.45391360103789591, 0.43879620409786158, 0.54657759325141297, 0.04167858076332176, 0.12577979271679429, 0.40518903067224954, 0.71558040896964403, 0.45611037584686187, 0.6550713894935507, 0.34577308824595554, 0.56896516296672017, 0.094968367949704202, 0.96158127455686837, 0.37399455861947206, 0.19603558011285294, 0.86241554243364094, 0.79570308173511406, 0.30138362455896273, 0.37604753058354889, 0.87434106717360005, 0.50366432467676336, 0.71552548446405972, 0.73475801687121478, 0.87580083667657338, 0.78036344022837911, 0.68386525788775088, 0.3089845285968

Как видно, на уровне 0.05 при множественной проверке гипотеза не отвергается